In [ ]:
from mltrainer import Trainer, TrainerSettings, ReportTypes, metrics
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
import torch

import torch.optim as optim
optimizer = optim.Adam

import sys 
import os
sys.path.append(os.path.abspath('../models'))
sys.path.append(os.path.abspath('../dev'))

In [ ]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()

batchsize = 32

streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()


In [ ]:
from pathlib import Path
from CNN import CNN

log_dir = Path("modellog").resolve()
if not log_dir.exists():
    log_dir.mkdir(parents=True)

accuracy = metrics.Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

model = CNN(units = 128)

settings = TrainerSettings(
    epochs=100,
    metrics=[accuracy],
    logdir=log_dir,
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    earlystop_kwargs={
        "save": False,
        "verbose": True,
        "patience": 100,
    },
    scheduler_kwargs={
        "factor": 0.5,
        "patience": 10
    }
)
adam = optim.Adam(model.parameters(), lr=0.01)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optimizer,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    )

In [ ]:
trainer.loop()